In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime as dt
import seaborn as sns
import os

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### **Data Processing**

#### Merge Datasets

In [4]:
path = "/content/drive/MyDrive/Data Science/Data Sets/GDA - Cyclistic Bike Share"

files = [file for file in os.listdir(path)]

all_months_data = pd.DataFrame()

for file in files:
  df = pd.read_csv("/content/drive/MyDrive/Data Science/Data Sets/GDA - Cyclistic Bike Share/"+file)
  all_months_data = pd.concat([all_months_data, df])

all_months_data.to_csv("all_data.csv", index= False)

#### Read in Updated Data

In [5]:
data_df = pd.read_csv("all_data.csv")
all_data = data_df
all_data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86.000,Lincoln Ave & Diversey Pkwy,152.000,41.896,-87.661,41.932,-87.659,member
1,5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503.000,Kosciuszko Park,499.000,41.924,-87.715,41.931,-87.724,member
2,5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142.000,Indiana Ave & Roosevelt Rd,255.000,41.895,-87.618,41.868,-87.623,member
3,2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216.000,Wood St & Augusta Blvd,657.000,41.903,-87.698,41.899,-87.672,member
4,27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125.000,Sheridan Rd & Lawrence Ave,323.000,41.890,-87.626,41.969,-87.655,casual


#### Augument data with addition columns

##### **Ride Duration**

In [6]:
all_data['started_at'] = pd.to_datetime(all_data['started_at'])
all_data['ended_at'] = pd.to_datetime(all_data['ended_at'])
all_data['Ride Duration'] = (all_data['ended_at'] - all_data['started_at']).astype('timedelta64[s]').astype(np.int64)

##### **Month**

In [7]:
all_data['Month'] = all_data['started_at'].dt.month

##### **Day of the Week**

In [8]:
# Mon - Sun: 0 - 6
all_data['Day of Week'] = all_data['started_at'].dt.weekday

##### **Hour**

In [9]:
all_data['Hour'] = all_data['started_at'].dt.hour

##### **Time of Day**

In [10]:
# Morning (6AM-12PM), Afternoon (12PM-4PM), Evening (4PM-10PM), Late Night(10PM-6AM)

def time_of_day(hour):
    if 6 <= hour < 12:
      return 'Morning'
    elif 12 <= hour < 16:
      return 'Afternoon'
    elif 16 <= hour < 20:
      return 'Evening'
    else: 
      return 'Late Night'

all_data['Time of Day'] = all_data['Hour']
all_data['Time of Day'] = all_data['Time of Day'].apply(time_of_day)

### **Data Cleaning**

#### Null Values

In [11]:
all_data.info(verbose=True, null_counts=True)

''' 
Null values only in:

start_station_name
start_station_id 
end_station_name
end_station_id
end_lat
end_lng

Rows still contains essential data and shouldn't be drop 
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3826978 entries, 0 to 3826977
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ride_id             3826978 non-null  object        
 1   rideable_type       3826978 non-null  object        
 2   started_at          3826978 non-null  datetime64[ns]
 3   ended_at            3826978 non-null  datetime64[ns]
 4   start_station_name  3678747 non-null  object        
 5   start_station_id    3678121 non-null  object        
 6   end_station_name    3655562 non-null  object        
 7   end_station_id      3655101 non-null  object        
 8   start_lat           3826978 non-null  float64       
 9   start_lng           3826978 non-null  float64       
 10  end_lat             3821973 non-null  float64       
 11  end_lng             3821973 non-null  float64       
 12  member_casual       3826978 non-null  object        
 13  Ride Duratio

" \nNull values only in:\n\nstart_station_name\nstart_station_id \nend_station_name\nend_station_id\nend_lat\nend_lng\n\nRows still contains essential data and shouldn't be drop \n"

#### Duplicates

In [12]:
# Check if any rows are complete duplicates
all_data.duplicated().sum()

0

In [13]:
# Main column to screen for duplicates for error is 'ride_id'
all_data.ride_id.duplicated().sum()

209

In [14]:
all_data.loc[all_data.ride_id.duplicated(keep=False), :].sort_values('ride_id').head(6)

# By sorting duplicate rows by 'ride_id', it shows how each pair includes a negative 'Ride Duration' which is invalid. 
# Seems like a system/data collection error

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Ride Duration,Month,Day of Week,Hour,Time of Day
3006508,021A73F8C18B932D,docked_bike,2020-12-15 12:15:58,2020-11-25 16:48:02,Clark St & Winnemac Ave,TA1309000035,Ravenswood Ave & Berteau Ave,TA1309000018,41.973,-87.668,41.958,-87.674,member,-1711676,12,1,12,Afternoon
2829583,021A73F8C18B932D,docked_bike,2020-11-25 16:35:39,2020-11-25 16:48:02,Clark St & Winnemac Ave,325.000,Ravenswood Ave & Berteau Ave,314.000,41.973,-87.668,41.958,-87.674,member,743,11,2,16,Evening
2797522,0334987B57662109,docked_bike,2020-11-25 16:15:04,2020-11-25 16:22:04,Broadway & Berwyn Ave,294.000,Lakefront Trail & Bryn Mawr Ave,459.000,41.978,-87.660,41.984,-87.652,member,420,11,2,16,Evening
3020384,0334987B57662109,docked_bike,2020-12-15 11:56:33,2020-11-25 16:22:04,Broadway & Berwyn Ave,13109,Lakefront Trail & Bryn Mawr Ave,KA1504000152,41.978,-87.660,41.984,-87.652,member,-1712069,12,1,11,Morning
3030159,038CAB4A84D9A56B,docked_bike,2020-12-15 11:57:41,2020-11-25 16:24:56,Wabash Ave & 9th St,TA1309000010,Wabash Ave & 9th St,TA1309000010,41.871,-87.626,41.871,-87.626,member,-1711965,12,1,11,Morning
2781321,038CAB4A84D9A56B,docked_bike,2020-11-25 16:08:05,2020-11-25 16:24:56,Wabash Ave & 9th St,321.000,Wabash Ave & 9th St,321.000,41.871,-87.626,41.871,-87.626,member,1011,11,2,16,Evening


In [15]:
# Find/drop all rows that are duplicates AND Negative 'Ride Duration'

all_data_filtered = all_data.drop(all_data.loc[(all_data.ride_id.duplicated(keep=False)) & (all_data['Ride Duration'] < 0) , :].index)

# Dropping 211 rows instead of 209/418 duplicates. 2 original rows had negative 'Ride Duration'

#### Negative Values *(Ride Duration)*

In [16]:
# Check for any negative values for 'Ride Duration'

all_data_filtered.loc[all_data_filtered['Ride Duration'] < 0, 'Ride Duration'].count()

10346

In [17]:
# Drop rows with negative values for 'Ride Duration'
all_data_filtered.drop(all_data_filtered.loc[all_data_filtered['Ride Duration'] < 0, :].index, inplace=True)

In [18]:
all_data_filtered.loc[all_data_filtered['Ride Duration'] < 0, 'Ride Duration'].count()

0

#### Outliers *(Ride Duration)*

##### **IQR Method**

In [19]:
Q1 = all_data_filtered['Ride Duration'].quantile(0.25)
Q3 = all_data_filtered['Ride Duration'].quantile(0.75)
IQR = Q3 - Q1
Upper_OL = (Q3 + (1.5 * IQR))
Lower_OL = (Q1 - (1.5 * IQR))

#Lower_OL is in negatives, doesn't apply

all_data_filtered.drop(all_data_filtered.loc[all_data_filtered['Ride Duration'] > Upper_OL, 'Ride Duration'].index, inplace=True)

### **Data Analysis**

#### Summary Statistics

In [20]:
all_data_filtered.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3518832 entries, 0 to 3826977
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ride_id             3518832 non-null  object        
 1   rideable_type       3518832 non-null  object        
 2   started_at          3518832 non-null  datetime64[ns]
 3   ended_at            3518832 non-null  datetime64[ns]
 4   start_station_name  3375479 non-null  object        
 5   start_station_id    3374878 non-null  object        
 6   end_station_name    3359962 non-null  object        
 7   end_station_id      3359515 non-null  object        
 8   start_lat           3518832 non-null  float64       
 9   start_lng           3518832 non-null  float64       
 10  end_lat             3516664 non-null  float64       
 11  end_lng             3516664 non-null  float64       
 12  member_casual       3518832 non-null  object        
 13  Ride Duratio

In [21]:
all_data_filtered.describe(include='object')

,ride_id,rideable_type,start_station_name,start_station_id,end_station_name,end_station_id,member_casual,Time of Day
count,3518832,3518832,3375479,3374878.000,3359962,3359515.000,3518832,3518832
unique,3518832,3,711,1923.000,712,1927.000,2,4
top,81E0B9735EA52FA6,docked_bike,Clark St & Elm St,176.000,Clark St & Elm St,35.000,member,Evening
freq,1,2325331,33323,25896.000,33201,26778.000,2223302,1230697


In [22]:
all_data_filtered['Ride Duration'].describe()

count   3518832.000
mean        978.910
std         693.933
min           0.000
25%         445.000
50%         790.000
75%        1362.000
max        3258.000
Name: Ride Duration, dtype: float64

#### Ride Count Analysis

##### **Total Ride Count** 

In [23]:
total_rc = all_data_filtered['ride_id'].count()

##### **Ride Count by Member Casual** 

In [24]:
mcrc_ovr = all_data_filtered.groupby('member_casual')['ride_id'].count().reset_index()
mcrc_ovr['%'] = mcrc_ovr['ride_id']/total_rc*100
mcrc_ovr

,member_casual,ride_id,%
0,casual,1295530,36.817
1,member,2223302,63.183


##### **Ride Count by Bike Type, Member Casual** 

In [25]:
mcrc_biketype = all_data_filtered.groupby(['member_casual', 'rideable_type'])['ride_id'].count().reset_index()
mcrc_biketype['%'] = mcrc_biketype['ride_id']/total_rc*100
mcrc_biketype

,member_casual,rideable_type,ride_id,%
0,casual,classic_bike,124731,3.545
1,casual,docked_bike,909738,25.853
2,casual,electric_bike,261061,7.419
3,member,classic_bike,389065,11.057
4,member,docked_bike,1415593,40.229
5,member,electric_bike,418644,11.897


##### **Ride Count by Time of Day, Member Casual** 

In [26]:
mcrc_TOD = all_data_filtered.groupby(['member_casual', 'Time of Day'])['ride_id'].count().reset_index()
mcrc_TOD['%'] = mcrc_TOD['ride_id']/total_rc*100
mcrc_TOD

,member_casual,Time of Day,ride_id,%
0,casual,Afternoon,386438,10.982
1,casual,Evening,454651,12.921
2,casual,Late Night,227690,6.471
3,casual,Morning,226751,6.444
4,member,Afternoon,600007,17.051
5,member,Evening,776046,22.054
6,member,Late Night,253979,7.218
7,member,Morning,593270,16.860


##### **Ride Count by Day of Week, Member Casual** 

In [27]:
mcrc_DOW = all_data_filtered.groupby(['member_casual', 'Day of Week'])['ride_id'].count().reset_index()
mcrc_DOW['%'] = mcrc_DOW['ride_id']/total_rc*100
mcrc_DOW

,member_casual,Day of Week,ride_id,%
0,casual,0,138914,3.948
1,casual,1,140558,3.994
2,casual,2,146415,4.161
3,casual,3,153390,4.359
4,casual,4,196692,5.590
5,casual,5,292981,8.326
6,casual,6,226580,6.439
7,member,0,291899,8.295
8,member,1,312956,8.894
9,member,2,327126,9.296


##### **Ride Count by Ride Hour, Member Casual** 

In [28]:
mcrc_hour = all_data_filtered.groupby(['member_casual', 'Hour'])['ride_id'].count().reset_index()
mcrc_hour['%'] = mcrc_hour['ride_id']/total_rc*100
mcrc_hour

,member_casual,Hour,ride_id,%
0,casual,0,19699,0.560
1,casual,1,12349,0.351
2,casual,2,6586,0.187
3,casual,3,3591,0.102
4,casual,4,3292,0.094
5,casual,5,5105,0.145
6,casual,6,12630,0.359
7,casual,7,22913,0.651
8,casual,8,30960,0.880
9,casual,9,37528,1.066


##### **Ride Count by Ride Month, Member Casual** 

In [29]:
mcrc_month = all_data_filtered.groupby(['member_casual', 'Month'])['ride_id'].count().reset_index()
mcrc_month['%'] = mcrc_month['ride_id']/total_rc*100
mcrc_month

,member_casual,Month,ride_id,%
0,casual,1,17024,0.484
1,casual,2,8994,0.256
2,casual,3,72207,2.052
3,casual,4,136635,3.883
4,casual,5,67403,1.915
5,casual,6,122079,3.469
6,casual,7,205467,5.839
7,casual,8,235368,6.689
8,casual,9,196196,5.576
9,casual,10,129021,3.667


#### Ride Duration Analysis

##### **Avg Ride Duration by Member Casual** 

In [30]:
mcrd_ovr = all_data_filtered.groupby('member_casual')['Ride Duration'].mean().reset_index()
mcrd_ovr['Ride Duration (min)'] = mcrd_ovr['Ride Duration']/60
mcrd_ovr

,member_casual,Ride Duration,Ride Duration (min)
0,casual,1195.783,19.930
1,member,852.538,14.209


##### **Avg Ride Duration by Bike Type, Member Casual** 

In [31]:
mcrd_biketype = all_data_filtered.groupby(['member_casual', 'rideable_type'])['Ride Duration'].mean().reset_index()
mcrd_biketype['Ride Duration (min)'] = mcrd_biketype['Ride Duration']/60
mcrd_biketype

,member_casual,rideable_type,Ride Duration,Ride Duration (min)
0,casual,classic_bike,1091.383,18.190
1,casual,docked_bike,1279.345,21.322
2,casual,electric_bike,954.471,15.908
3,member,classic_bike,774.216,12.904
4,member,docked_bike,908.465,15.141
5,member,electric_bike,736.214,12.270


##### **Avg Ride Duration by Time of Day, Member Casual** 

In [32]:
mcrd_TOD = all_data_filtered.groupby(['member_casual', 'Time of Day'])['Ride Duration'].mean().reset_index()
mcrd_TOD['Ride Duration (min)'] = mcrd_TOD['Ride Duration']/60
mcrd_TOD

,member_casual,Time of Day,Ride Duration,Ride Duration (min)
0,casual,Afternoon,1250.327,20.839
1,casual,Evening,1204.607,20.077
2,casual,Late Night,1172.326,19.539
3,casual,Morning,1108.688,18.478
4,member,Afternoon,877.974,14.633
5,member,Evening,882.339,14.706
6,member,Late Night,788.519,13.142
7,member,Morning,815.236,13.587


##### **Avg Ride Duration by Day of Week, Member Casual** 

In [33]:
mcrd_DOW = all_data_filtered.groupby(['member_casual', 'Day of Week'])['Ride Duration'].mean().reset_index()
mcrd_DOW['Ride Duration (min)'] = mcrd_DOW['Ride Duration']/60
mcrd_DOW

,member_casual,Day of Week,Ride Duration,Ride Duration (min)
0,casual,0,1177.755,19.629
1,casual,1,1143.698,19.062
2,casual,2,1118.565,18.643
3,casual,3,1115.648,18.594
4,casual,4,1154.034,19.234
5,casual,5,1268.190,21.136
6,casual,6,1285.910,21.432
7,member,0,820.380,13.673
8,member,1,818.152,13.636
9,member,2,816.598,13.610


##### **Avg Ride Duration by Ride Month, Member Casual** 

In [34]:
mcrd_month = all_data_filtered.groupby(['member_casual', 'Month'])['Ride Duration'].mean().reset_index()
mcrd_month['Ride Duration (min)'] = mcrd_month['Ride Duration']/60
mcrd_month

,member_casual,Month,Ride Duration,Ride Duration (min)
0,casual,1,892.452,14.874
1,casual,2,1044.324,17.405
2,casual,3,1142.623,19.044
3,casual,4,1150.614,19.177
4,casual,5,1375.128,22.919
5,casual,6,1318.827,21.980
6,casual,7,1312.520,21.875
7,casual,8,1235.620,20.594
8,casual,9,1155.497,19.258
9,casual,10,1049.467,17.491


##### **Avg Ride Duration by Ride Hour, Member Casual** 

In [35]:
mcrd_hour = all_data_filtered.groupby(['member_casual', 'Hour'])['Ride Duration'].mean().reset_index()
mcrd_hour['Ride Duration (min)'] = mcrd_hour['Ride Duration']/60
mcrd_hour

,member_casual,Hour,Ride Duration,Ride Duration (min)
0,casual,0,1185.131,19.752
1,casual,1,1231.247,20.521
2,casual,2,1256.447,20.941
3,casual,3,1144.740,19.079
4,casual,4,1046.907,17.448
5,casual,5,930.868,15.514
6,casual,6,926.652,15.444
7,casual,7,974.724,16.245
8,casual,8,1002.915,16.715
9,casual,9,1076.374,17.940


#### Top Start/End Stations

##### **Top 10 Start Station - Overall** 

In [36]:
start_station_ovr = all_data_filtered.groupby('start_station_name')['ride_id'].count().reset_index().sort_values('ride_id', ascending=False)
start_station_ovr['%'] = start_station_ovr['ride_id']/total_rc*100
start_station_ovr.head(10)

,start_station_name,ride_id,%
139,Clark St & Elm St,33323,0.947
615,Streeter Dr & Grand Ave,30984,0.881
618,Theater on the Lake,29026,0.825
372,Lake Shore Dr & Monroe St,27038,0.768
657,Wells St & Concord Ln,26554,0.755
373,Lake Shore Dr & North Blvd,26118,0.742
57,Broadway & Barry Ave,24743,0.703
658,Wells St & Elm St,24711,0.702
215,Dearborn St & Erie St,24451,0.695
133,Clark St & Armitage Ave,24179,0.687


##### **Top 10 End Station - Overall** 

In [37]:
end_station_ovr = all_data_filtered.groupby('end_station_name')['ride_id'].count().reset_index().sort_values('ride_id', ascending=False)
end_station_ovr['%'] = end_station_ovr['ride_id']/total_rc*100
end_station_ovr.head(10)

,end_station_name,ride_id,%
138,Clark St & Elm St,33201,0.944
615,Streeter Dr & Grand Ave,31963,0.908
618,Theater on the Lake,29735,0.845
657,Wells St & Concord Ln,27129,0.771
372,Lake Shore Dr & North Blvd,26109,0.742
371,Lake Shore Dr & Monroe St,25945,0.737
56,Broadway & Barry Ave,25396,0.722
214,Dearborn St & Erie St,24975,0.710
584,St. Clair St & Erie St,24040,0.683
658,Wells St & Elm St,23446,0.666


##### **Top 10 Start Station - Member Casual**

In [38]:
mc_start_station = all_data_filtered.groupby(['member_casual', 'start_station_name'])['ride_id'].count().reset_index()
mc_start_station['%'] = mc_start_station['ride_id']/total_rc*100
mc_start_station.sort_values(['member_casual', 'ride_id'], ascending=False).groupby('member_casual').head(10)

,member_casual,start_station_name,ride_id,%
836,member,Clark St & Elm St,21792,0.619
754,member,Broadway & Barry Ave,16818,0.478
1277,member,St. Clair St & Erie St,16620,0.472
1348,member,Wells St & Concord Ln,16555,0.470
911,member,Dearborn St & Erie St,16545,0.470
1049,member,Kingsbury St & Kinzie St,15657,0.445
1310,member,Theater on the Lake,15447,0.439
1349,member,Wells St & Elm St,15428,0.438
1352,member,Wells St & Huron St,15277,0.434
830,member,Clark St & Armitage Ave,14832,0.422


##### **Top 10 End Station - Member Casual**

In [39]:
mc_end_station = all_data_filtered.groupby(['member_casual', 'end_station_name'])['ride_id'].count().reset_index()
mc_end_station['%'] = mc_end_station['ride_id']/total_rc*100
mc_end_station.sort_values(['member_casual', 'ride_id'], ascending=False).groupby('member_casual').head(10)

,member_casual,end_station_name,ride_id,%
848,member,Clark St & Elm St,22318,0.634
1288,member,St. Clair St & Erie St,17828,0.507
924,member,Dearborn St & Erie St,17105,0.486
766,member,Broadway & Barry Ave,17068,0.485
1359,member,Wells St & Concord Ln,16986,0.483
1061,member,Kingsbury St & Kinzie St,15822,0.450
1322,member,Theater on the Lake,15058,0.428
1360,member,Wells St & Elm St,14666,0.417
1363,member,Wells St & Huron St,14322,0.407
1348,member,Wabash Ave & Roosevelt Rd,14150,0.402


### **Data Visualization (External)**

Will be done using Tableau with extracted CSV file

In [40]:
all_data_filtered.to_csv('GDA_Cyclistic_Python_Final')